# Lesson-12 CNN practice

# Deep learning

Previously in 2_fullyconnected.ipynb and 3_regularization.ipynb, we trained fully connected networks to classify notMNIST characters.
The goal of this assignment is make the neural network convolutional.

In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
path = '/home/student/project/project-01/nlp_bots/hw-xujing'
pickle_file = 'notMNIST.pickle'
pickle_file1 = 'notMNIST_clean.pickle'

In [3]:
import os
with open(os.path.join(path, pickle_file), 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save
    print('training set', train_dataset.shape, train_labels.shape)
    print('valid set', valid_dataset.shape, valid_labels.shape)
    print('test set', test_dataset.shape, test_labels.shape)

training set (200000, 28, 28) (200000,)
valid set (10000, 28, 28) (10000,)
test set (10000, 28, 28) (10000,)


reformat into tensorflow friendly shape
1. convolutions need the image data formated as cube(width by height by #channels)
2. labels as float 1-hot encoding

In [4]:
#a = np.array([[1, 2], [3, 4]],[[5, 6], [7, 8]])
a = np.array([[[1,2],[3,4]],[[5,6],[7,8]]])
b = a.reshape(-1, 2, 2, 1)
print(a.shape)
print(b.shape)

(2, 2, 2)
(2, 2, 2, 1)


In [5]:
print(a)
print(b)

[[[1 2]
  [3 4]]

 [[5 6]
  [7 8]]]
[[[[1]
   [2]]

  [[3]
   [4]]]


 [[[5]
   [6]]

  [[7]
   [8]]]]


In [4]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size, image_size, num_channels)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [5]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
           / predictions.shape[0])

In [6]:
# valid parameter out of the range of windows, remove the data, SAME is the over part, fill 0
#input1 = tf.Variable(tf.reshape(tf.range(64), (1, 8, 8, 1)))
input1 = tf.Variable(tf.ones([1, 224, 224, 1]))#num-inputs, inputs-size, inputs-size, num_channels
filter1 = tf.Variable(tf.ones([11, 11, 1, 1]))#kernel-size, kernel-size, num-channels, depth
op = tf.nn.conv2d(input1, filter1, [1, 1, 1, 1], padding='SAME')
# filter parameter is the window paramenter, [1, 1, 1, 1] is the stride parameter
#op = tf.nn.max_pool(input1, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')

In [7]:
with tf.Session() as ses:
    ses.run(tf.global_variables_initializer())
    res = ses.run(op)
    print(res.shape)
    output = tf.reshape(res, [res.shape[1], res.shape[1]])
    #print(ses.run(output))

(1, 224, 224, 1)


In [6]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():
    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    #patch_size核函数大小，depth表示核函数个数，代表会出现多少个卷积特征图像;
    layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    
    layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    
    layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

    # Model.
    def model(data):
        #卷积操作，strides[1]，strides[2]决定卷积核在输入图像in_hight，in_width方向的滑动步长
        conv = tf.nn.conv2d(data,layer1_weights, strides = [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases

    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels = tf_train_labels, logits= logits))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [7]:
num_steps = 1001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 100 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
            valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 3.175600
Minibatch accuracy: 6.2%
Validation accuracy: 10.8%
Minibatch loss at step 100: 1.482311
Minibatch accuracy: 56.2%
Validation accuracy: 72.1%
Minibatch loss at step 200: 0.496939
Minibatch accuracy: 75.0%
Validation accuracy: 75.0%
Minibatch loss at step 300: 0.649083
Minibatch accuracy: 75.0%
Validation accuracy: 76.2%
Minibatch loss at step 400: 0.175056
Minibatch accuracy: 100.0%
Validation accuracy: 79.7%
Minibatch loss at step 500: 0.818376
Minibatch accuracy: 75.0%
Validation accuracy: 81.0%
Minibatch loss at step 600: 0.405247
Minibatch accuracy: 87.5%
Validation accuracy: 81.0%
Minibatch loss at step 700: 0.255050
Minibatch accuracy: 87.5%
Validation accuracy: 81.5%
Minibatch loss at step 800: 0.243011
Minibatch accuracy: 100.0%
Validation accuracy: 82.2%
Minibatch loss at step 900: 0.564435
Minibatch accuracy: 81.2%
Validation accuracy: 82.9%
Minibatch loss 

# Problem 1

the convolutional model above used the convolutions with stride 2 to reduce the dimentionality, replace the strides by using a max pooling operation with strides 2 and kernels 2.

In [7]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()
with graph.as_default():
    
    # input data
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    #variables
    layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_bias = tf.Variable(tf.zeros([depth]))
    
    layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_bias = tf.Variable(tf.constant(1.0, shape=[depth]))
    
    layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_bias = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    layer4_bias = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
    #model
    def model(data):
        conv1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
        bias1 = tf.nn.relu(conv1 + layer1_bias)
        pool1 = tf.nn.max_pool(bias1, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
        
        conv2 = tf.nn.conv2d(pool1, layer2_weights, [1, 1, 1, 1], padding='SAME')
        bias2 = tf.nn.relu(conv2 + layer2_bias)
        pool2 = tf.nn.max_pool(bias2, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
        
        shape = pool2.get_shape().as_list()
        reshape = tf.reshape(pool2, [shape[0], shape[1] *shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_bias)
        return tf.matmul(hidden, layer4_weights) +layer4_bias
    
    #training computation
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))
    
    #optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
    
    #prediction for training validation , test
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))
        

In [8]:
num_steps = 5001
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] -batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 100 == 0):
            print('minibatch loss at step %d: %f' % (step, l))
            print('minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('validation accuracy %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('test accuracy %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
minibatch loss at step 0: 4.200749
minibatch accuracy: 12.5%
validation accuracy 10.0%
minibatch loss at step 100: 1.920781
minibatch accuracy: 43.8%
validation accuracy 62.0%
minibatch loss at step 200: 0.586590
minibatch accuracy: 81.2%
validation accuracy 73.3%
minibatch loss at step 300: 0.524761
minibatch accuracy: 75.0%
validation accuracy 79.7%
minibatch loss at step 400: 0.232351
minibatch accuracy: 93.8%
validation accuracy 79.8%
minibatch loss at step 500: 0.725909
minibatch accuracy: 75.0%
validation accuracy 82.1%
minibatch loss at step 600: 0.361743
minibatch accuracy: 87.5%
validation accuracy 82.4%
minibatch loss at step 700: 0.183780
minibatch accuracy: 100.0%
validation accuracy 83.3%
minibatch loss at step 800: 0.344542
minibatch accuracy: 81.2%
validation accuracy 83.6%
minibatch loss at step 900: 0.465809
minibatch accuracy: 87.5%
validation accuracy 84.1%
minibatch loss at step 10

# Problem 2

Try to get the best performance you can using a convolutional net. Look for example at the classic LeNet5 architecture, adding Dropout, and/or adding learning rate decay.

In [10]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()
with graph.as_default():
    # input data
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape= (batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    #variables 
    layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_bias = tf.Variable(tf.zeros([depth]))
    
    layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_bias = tf.Variable(tf.constant(1.0, shape=[depth]))
    
    size3 = ((image_size - patch_size + 1) // 2 - patch_size + 1) // 2
    layer3_weights = tf.Variable(tf.truncated_normal([size3 * size3 * depth, num_hidden], stddev=0.1))
    layer3_bias = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    layer4_bias = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
    #model
    def model(data):
        #C1 input 28 * 28
        conv1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='VALID')
        bias1 = tf.nn.relu(conv1 + layer1_bias)
        #S2 input 24 * 24 (28 -5)/1 + 1 =24
        pool2 = tf.nn.avg_pool(bias1, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
        #C3 input 12 * 12 (24-2)/2+1 = 12
        conv3 = tf.nn.conv2d(pool2, layer2_weights, [1, 1, 1, 1], padding='VALID')
        bias3 = tf.nn.relu(conv3 + layer2_bias)
        #S4 input 8 * 8  (12-5)/1+1 = 8
        pool4 = tf.nn.avg_pool(bias3, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
        #F6 input 4 * 4   (8-2)/2+1 = 4
        shape = pool4.get_shape().as_list()
        reshape = tf.reshape(pool4, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_bias)
        return tf.matmul(hidden, layer4_weights) + layer4_bias
    
    #training
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
    #optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
    
    #predictions
    train_prdiction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))
        

In [12]:
num_steps = 10001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prdiction], feed_dict=feed_dict)
        if (step % 200 == 0):
            print('minibatch loss at step %d: %f' % (step, l))
            print('minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
minibatch loss at step 0: 2.495801
minibatch accuracy: 6.2%
validation accuracy: 9.9%
minibatch loss at step 200: 0.868350
minibatch accuracy: 75.0%
validation accuracy: 71.9%
minibatch loss at step 400: 0.288443
minibatch accuracy: 100.0%
validation accuracy: 77.0%
minibatch loss at step 600: 0.442672
minibatch accuracy: 87.5%
validation accuracy: 78.2%
minibatch loss at step 800: 0.352698
minibatch accuracy: 93.8%
validation accuracy: 80.8%
minibatch loss at step 1000: 0.798899
minibatch accuracy: 75.0%
validation accuracy: 80.6%
minibatch loss at step 1200: 0.368952
minibatch accuracy: 81.2%
validation accuracy: 82.7%
minibatch loss at step 1400: 0.660902
minibatch accuracy: 87.5%
validation accuracy: 82.7%
minibatch loss at step 1600: 0.329128
minibatch accuracy: 93.8%
validation accuracy: 83.3%
minibatch loss at step 1800: 0.719280
minibatch accuracy: 68.8%
validation accuracy: 84.1%
minibatch loss at step 2000: 0.157120
minibatch accuracy: 93.8%
validation accuracy: 8

using dropout and learning rate decay in next model

In [13]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden =64
beta_regul = 1e-3
drop_out = 0.5

graph = tf.Graph()
with graph.as_default():
    #input data
    tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    global_step = tf.Variable(0)
    
    #Variable
    layer1_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_bias = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_bias = tf.Variable(tf.constant(1.0, shape=[depth]))
    size3 = ((image_size - patch_size +1) // 2 - patch_size + 1) //2
    layer3_weights = tf.Variable(tf.truncated_normal(
    [size3 * size3 * depth, num_hidden], stddev=0.1))
    layer3_bias = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal(
    [num_hidden, num_hidden], stddev=0.1))
    layer4_bias = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer5_weights = tf.Variable(tf.truncated_normal(
    [num_hidden, num_labels], stddev=0.1))
    layer5_bias = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
    #model
    def model(data, keep_prob):
        #C1 input 28 * 28
        conv1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='VALID')
        bias1 = tf.nn.relu(conv1 + layer1_bias)
        #S2 input 24 * 24
        pool2 = tf.nn.avg_pool(bias1, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
        #C3 input 12 * 12
        conv3 = tf.nn.conv2d(pool2, layer2_weights, [1, 1, 1, 1], padding='VALID')
        bias3 = tf.nn.relu(conv3 + layer2_bias)
        #S4 input 8 * 8
        pool4 = tf.nn.avg_pool(bias3, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
        #F5 input 4 * 4
        shape = pool4.get_shape().as_list()
        reshape = tf.reshape(pool4, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden5 = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_bias)
        #F6
        drop5 = tf.nn.dropout(hidden5, keep_prob)
        hidden6 = tf.nn.relu(tf.matmul(hidden5, layer4_weights) + layer4_bias)
        drop6 = tf.nn.dropout(hidden6, keep_prob)
        return tf.matmul(drop6, layer5_weights) + layer5_bias
    
    #training computation
    logits = model(tf_train_dataset, drop_out)
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))
    
    #optimizer
    learning_rate = tf.train.exponential_decay(0.05, global_step, 1000, 0.85, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    #predictions
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset, 1.0))
    test_prediction = tf.nn.softmax(model(tf_test_dataset, 1.0))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [14]:
 num_steps = 5001
    
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 100 == 0):
            print('minibatch loss at step %d: %f' % (step, l))
            print('minibatch accuracy : %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
minibatch loss at step 0: 2.926104
minibatch accuracy : 18.8%
Validation accuracy: 10.0%
minibatch loss at step 100: 1.958172
minibatch accuracy : 25.0%
Validation accuracy: 49.7%
minibatch loss at step 200: 1.184326
minibatch accuracy : 50.0%
Validation accuracy: 61.2%
minibatch loss at step 300: 1.057638
minibatch accuracy : 56.2%
Validation accuracy: 69.4%
minibatch loss at step 400: 0.620198
minibatch accuracy : 87.5%
Validation accuracy: 70.7%
minibatch loss at step 500: 1.158337
minibatch accuracy : 68.8%
Validation accuracy: 75.7%
minibatch loss at step 600: 0.482990
minibatch accuracy : 87.5%
Validation accuracy: 75.0%
minibatch loss at step 700: 0.996806
minibatch accuracy : 68.8%
Validation accuracy: 77.9%
minibatch loss at step 800: 0.829691
minibatch accuracy : 81.2%
Validation accuracy: 79.1%
minibatch loss at step 900: 0.620502
minibatch accuracy : 81.2%
Validation accuracy: 79.2%
minibatch loss at step 1000: 0.952111
minibatch accuracy : 75.0%
Validation accu